# Parcial 2
### **Elaboró: Anette Paulina González Solano**

In [1]:
import scipy.io # Leer .mat
import pandas as pd # DataFrames y percentiles
import networkx as nx # Análisis de redes, métricas
import numpy as np # Operaciones numéricas y matrices
import seaborn as sns # Mapas de calor, histogramas
import matplotlib.pyplot as plt # Gráficas 2D y 3D

In [22]:
# Cargar archivo .mat
mat = scipy.io.loadmat('Coactivation_matrix.mat')
# Extraer variables relevantes
coact_matrix = mat['Coactivation_matrix'] # Matriz de coactivación (638 x 638)
coords = mat['Coord'] # Coordenadas 3D de los nodos (638 x 3)
# Crear grafo de 'coact_matrix' (es ponderado y no dirigido)
G = nx.from_numpy_array(coact_matrix)

In [16]:
# (Solo para visualizar datos) DataFrame de 'coact_matrix'
df_coact_matrix = pd.DataFrame(coact_matrix)
# (Solo para visualizar datos) DataFrame de 'coords'
df_coords = pd.DataFrame(coords)

df_coact_matrix, df_coords

(          0         1         2         3         4        5         6    \
 0    0.000000  0.160714  0.111486  0.000000  0.138095  0.00000  0.000000   
 1    0.160714  0.000000  0.068259  0.000000  0.042056  0.00000  0.000000   
 2    0.111486  0.068259  0.000000  0.224199  0.000000  0.00000  0.000000   
 3    0.000000  0.000000  0.224199  0.000000  0.000000  0.00000  0.039275   
 4    0.138095  0.042056  0.000000  0.000000  0.000000  0.00000  0.000000   
 ..        ...       ...       ...       ...       ...      ...       ...   
 633  0.053872  0.042105  0.161905  0.000000  0.000000  0.00000  0.000000   
 634  0.049383  0.000000  0.103943  0.075697  0.000000  0.03876  0.000000   
 635  0.000000  0.000000  0.034130  0.108787  0.000000  0.00000  0.105839   
 636  0.050459  0.000000  0.000000  0.000000  0.103448  0.00000  0.000000   
 637  0.101124  0.069231  0.000000  0.000000  0.053279  0.00000  0.000000   
 
           7    8         9    ...  628       629  630  631       632  \
 

**1. Utilizando el dataset de _Coactivation_matrix.mat_:**
   
    1.1 Calcule el coeficiente de mundo pequeño

    1.2 Calcule las comunidades del grafo

    1.3 Calcule los hub

    1.4 Calcule la eficiencia global

    1.5 Calcule la eficiencia local

    1.6 Determine el grado de cada nodo

In [41]:
# 1.1 Coeficiente de mundo pequeño
# Mundo pequeño: ↑ coeficiente de clustering + ↓ longitud promedio

# Coeficiente de clustering: ¿Qué tan conectados están los vecinos de un nodo entre sí?
clustering = nx.average_clustering(G)
# Longitud promedio de los caminos más cortos entre nodos: ¿Cuántos pasos en promedio se necesitan para ir de un nodo a otro? (valor bajo = red eficiente)
path_length = nx.average_shortest_path_length(G)

print("Coeficiente de clustering:", clustering)
print("Longitud promedio de caminos más cortos:", path_length)
# Habemus mundo pequeño

Coeficiente de clustering: 0.3844533292242753
Longitud promedio de caminos más cortos: 2.2148737961545844


In [27]:
# 1.2 Comunidades del grafo
# Comunidad: grupo de nodos más conectados entre sí que con el resto de nodos (i.e., posibles regiones funcionales)

# Importar módulo de detección de comunidades (networkx)
from networkx.algorithms import community

# Detectar comunidades en G usando Louvain
comunidades = community.louvain_communities(G, weight='weight')
print("Número de comunidades:", len(comunidades))
for i, comunidad in enumerate(comunidades):
    print(f"Comunidad {i+1}: {len(comunidad)} nodos")
# Calcular modularidad: ¿Qué tan bien conectados están los nodos dentro de su comunidad? (~1 = muy fuerte)
modularidad = community.modularity(G, comunidades, weight='weight')
print("Modularidad:", modularidad)

Número de comunidades: 5
Comunidad 1: 189 nodos
Comunidad 2: 138 nodos
Comunidad 3: 153 nodos
Comunidad 4: 56 nodos
Comunidad 5: 102 nodos
Modularidad: 0.4740204321779562


In [34]:
# 3. Hub: nodo con ↑ grado + ↑ centralidad de intermediación + ↑ centralidad de cercanía

# Grado: ¿Cuántas aristas tiene cada nodo?
grado = dict(G.degree())
# Centralidad de intermediación: ¿Cuántos caminos más cortos incluyen a un nodo?
intermediación = nx.betweenness_centrality(G)
# Centralidad de cercanía: ¿Qué tan cerca está un nodo de todos los demás nodos en el grafo?
cercanía = nx.closeness_centrality(G)

# Combinar medidas normalizadas en un “hub score”
hub_score = {n: (grado[n]/max(grado.values()) + # grado[n]/max(grado.values()) divide cada nodo por el grado máximo de la red
                 intermediación[n]/max(intermediación.values()) +
                 cercanía[n]/max(cercanía.values()))/3
             for n in G.nodes()} # Recorre todos los nodos del grafo G

# Top 10 hubs
top_10_hubs = sorted(hub_score.items(), key=lambda x: x[1], reverse=True)[:10] # hub_score.items() obtiene pares (nodo, puntaje)

print("Top 10 hubs:")
for n, score in top_10_hubs:
    print(n, round(score, 4))

Top 10 hubs:
330 0.9977
482 0.917
230 0.9021
416 0.8865
621 0.8844
235 0.8832
629 0.8656
485 0.8474
418 0.8206
38 0.8014


In [54]:
# 1.6 Grado de cada nodo

# Ordenar grados descendentemenre 
grado_ordenado = sorted(grado.items(), key=lambda x: x[1], reverse=True)
print(f"{'Nodo':<10} {'Grado':<5}")  # encabezado
print("-"*16)                      
for nodo, g in grado_ordenado:
    print(f"{nodo:<10} {g:<5}")

330        179  
482        172  
230        164  
416        163  
485        161  
235        160  
356        153  
494        152  
350        151  
400        151  
328        150  
491        150  
38         149  
488        147  
496        147  
481        143  
629        143  
418        142  
421        142  
121        141  
346        140  
622        140  
397        136  
621        136  
237        135  
286        131  
202        129  
630        129  
231        128  
44         126  
353        126  
7          124  
16         124  
275        124  
452        124  
465        124  
43         123  
500        123  
331        122  
135        121  
334        121  
42         120  
100        120  
97         119  
327        119  
405        118  
407        117  
70         115  
126        115  
130        115  
193        115  
344        115  
495        115  
623        115  
41         114  
399        114  
365        113  
124        112  
345        111

In [39]:
# 1.4 Eficiencia global: ¿Qué tan eficiente es la red para transmitir información?
ef_global = nx.global_efficiency(G)
print("Eficiencia global:", ef_global) # (~1 = mejor eficiencia)

Eficiencia global: 0.49492420551600974


In [42]:
# 1.5 Eficiencia local: ¿Qué tan eficiente es la comunicación entre los vecinos de cada nodo?
# Cerebro sano: ↑ eficiencia global y local
ef_local = nx.local_efficiency(G)
print("Eficiencia local:", ef_local) # (~1 = mejor eficiencia)

Eficiencia local: 0.6643934695960994


**2. Utilizando el dataset de _Coactivation_matrix.mat_:**
   
    2.1 Generar el mapa de calor de cada matriz de conectividad
   
    2.2 Generar la distribución de datos de cada matriz de conectividad
   
    2.3 ¿Qué valor se encuentra en el percentil 0.25, 0.5 y 0.75 de la matriz de conectividad?
   
    2.4 Cree los 3 grafos 2D filtrando la matriz de conectividad con los valores dados por los percentiles del ejercicio anterior

**3. Utilizando el dataset de _Coactivation_matrix.mat_:**

    3.1 Genere el grafo 3D
  
    3.2 Haga que el tamaño de nodos sea proporcional a su grado; es decir, q entre mayor sea su grado, mayor sea el tamaño del nodo ploteado

    3.3 Haga que el color de las aristas este relacionado al valor de la matriz de conectividad. Utilizar el map color Hot

In [4]:
# Cargar archivo .edf

**4. Utilizando el dataset de _chb01_01.edf_:**
    
    4.1 Calcule el coeficiente de mundo pequeño

    4.2 Calcule las comunidades del grafo

    4.3 Calcule los hub

    4.4 Calcule la eficiencia global

    4.5 Calcule la eficiencia local

    4.6 Determine el grado de cada nodo

**5. Utilizando el dataset de _chb01_01.edf_:**

    5.1 Genere el grafo 3D

    5.2 Haga que el tamaño de nodos sea proporcional a su grado; es decir, q entre mayor sea su grado, mayor sea el tamaño del nodo ploteado

    5.3 Haga que el color de las aristas este relacionado al valor de la matriz de conectividad. Utilizar el map color Hot